## DAY1 Renaming (.sh)

In [ ]:
#!/bin/bash

renamed_folders_count=0
inconsistent_format_count=0
date_extraction_failed_count=0


# Extract date from edf file 
extract_date() {
    local file="$1"
    local date_info=$(strings "$file" | grep "^DATE:" | awk -F "DATE:" '{print $2}')
    local month=$(echo "$date_info" | awk '{print $2}')
    local day=$(echo "$date_info" | awk '{print $3}')
    local year=$(echo "$date_info" | awk '{print $NF}')

    # Convert month 
    local month_num=$(date -d "$month 1 2000" +%m 2> /dev/null | sed 's/^0*//')

    # Format
    if [[ -z $month_num ]]; then
        echo "Error: Unable to extract date from $file."
        return 1
    else
        printf -v date_str "%4d%02d%02d" "$year" "$month_num" "$day"
        echo "$date_str"
    fi
}

# Rename files within session directory
rename_files() {
    local session_dir="$1"
    local id="$2"
    local date="$3"

    # Rename edf file
    local edf_file=$(find "$session_dir" -maxdepth 1 -name "*.edf")
    if [[ -n $edf_file ]]; then
        mv "$edf_file" "${session_dir}/sub-${id}_${date}_resting_fixation.edf"
    fi

    # Rename the "actual_Trial" .dat file
    local trial_dat_file=$(find "$session_dir" -maxdepth 1 -name "actual_Trial*.dat")
    if [[ -n $trial_dat_file ]]; then
        mv "$trial_dat_file" "${session_dir}/sub-${id}_${date}_resting_trial_fixation.dat"
    fi

    # Rename the "actual_firsttrialFixation" .dat file
    local first_trial_dat_file=$(find "$session_dir" -maxdepth 1 -name "actual_firsttrialFixation*.dat")
    if [[ -n $first_trial_dat_file ]]; then
        mv "$first_trial_dat_file" "${session_dir}/sub-${id}_${date}_resting_first_fixation.dat"
    fi
}

for id_folder in */; do
    if [[ -d "$id_folder" ]]; then
        id=${id_folder%/}
        for session_folder in "${id_folder}"*/; do
            session_name=$(basename "$session_folder")
            # Check alphanumeric characters
            if [[ $session_name =~ ^[a-zA-Z0-9]+$ ]]; then
                # Check if the session folder has already been renamed
                if ! [[ $session_name =~ ^[0-9]{8}$ ]]; then
                    # Find the .edf file to get the date
                    date=""
                    for edf_file in "${session_folder}"*.edf; do
                        if [[ -f "$edf_file" ]]; then
                            date=$(extract_date "$edf_file")
                            if [[ $? -eq 0 ]]; then
                                new_session_folder="${id_folder}${date}"
                                if [[ ! -d "$new_session_folder" ]]; then
                                    mv "$session_folder" "$new_session_folder" && {
                                        ((renamed_folders_count++))
                                        echo " $session_folder -> $new_session_folder"
                                        rename_files "$new_session_folder" "$id" "$date"
                                    }
                                fi
                                break
                           else
                                echo Error: Cannot find date for directory $session_folder
                                ((date_extraction_failed_count++))
                                break
                            fi
                        fi
                    done
                    if [[ -z $date ]]; then
                        echo Error: No .edf file with date found in $session_folder
                        ((date_extraction_failed_count++))
                    fi
                fi
            else
                echo Skipped renaming $session_folder due to inconsistent naming format \(non-alphanumeric characters present\).
                ((inconsistent_format_count++))
            fi
        done
    fi
done


# Print summary
echo Renaming completed.
echo Total session folders renamed: $renamed_folders_count
echo Total session folders skipped due to inconsistent format: $inconsistent_format_count
echo Total session folders not renamed due to date extraction failure: $date_extraction_failed_count